# Types of Enzyme Inhibition

## Table of Contents

1. [How to Use this Notebook](#how-to-use)
2. [Model Code](#model-code)
3. [Competitive Inhibition](#competitive)
4. [Uncompetitive Inhibition](#uncompetitive)
5. [Noncompetitive Inhibition](#noncompetitive)
6. [Key Observations](#key-observations)

<a id='how-to-use'></a>

## How to Use This Notebook

This notebook (a companion to the Walk in th Forest post, "[Types of Enzyme Inhibition](https://walkintheforest.com/post/types-of-enzyme-inhibition)"  provides an interactive graphing environment to explore the three types of inhibition and how $K_I$ and $[S]$ affect the Michaelis-Menten and Lineweaver Burk plots. When working with the interactive graphs, the most important things to observe are the general patterns of change rather than the specific values:

- How does changing $K_I$ affect the importance of $[I]$?
- How do each of the Lineweaver-Burk Plots change with $[I]$?


1. Select the "Kernel" menu button
2. Select "Restart and Run All"
3. Use the sliders to control $K_I$ and $[I]$ (Change the slider slowly to allow the graphs to update properly)
4. Observe how altering these values affects the Michaelis-Menten and Lineweaver-Burk plots
5. Compare your observations to those at the the bottom of the notebook.

### Known Bugs
- Sometimes the Lineweaver Burk plot traces curve back to the origin. Rerun the cell to reset the plots

In [1]:
# Imports #
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.tools as ptools
import pandas as pd
from plotly.subplots import make_subplots
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

<a id='model-code'></a>

## Model Code
Below are the functions and primary code to generate the graphs and interactive elements of the notebook. **You do not need to edit this code**. However, if you are interested in how the code works, comments have been provided to help step through each function.

In [2]:
### Plotting function
def plot_inhibition(title, mm_func, lb_func):
    """ Generates the initial plot (widget) for graphing
    
    Arguments:
        title: String of the inhibition type
        mm_func: Function to evaluate the Michaelis-Menten equation
        lb_func: Function to evaluate the Lineweaver-Burk equation
    
    Returns:
        mm_vs_lb: Plotly widget/plot of MM and LB side-by-side
    """
    
    # Generate the intial figure and starting traces
    mm_vs_lb = go.FigureWidget(
        make_subplots(rows=1, cols=2,
                      subplot_titles=("Michaelis-Menten Model",
                                      "Lineweaver-Burk Transformation")))
    mm_vs_lb.add_scatter(x=sub_pos, y=mm_func(ki, inhib_conc),
                     mode='lines', line_color='red', row=1, col=1,
                     showlegend=False)
    mm_vs_lb.add_scatter(x=1/substrate, y=lb_func(ki, inhib_conc),
                         mode='lines', line_color='blue', row=1, col=2,
                         showlegend=False)
    
    ## Update MM Plot Axes
    mm_vs_lb.update_xaxes(range=[0,conc_high], row=1, col=1)
    mm_vs_lb.update_yaxes(range=[0,20], row=1, col=1)

    ## Update LB Plot Axes
    mm_vs_lb.update_xaxes(range=[-1,1], row=1, col=2)
    mm_vs_lb.update_yaxes(range=[-1,1], row=1, col=2)
    mm_vs_lb.update_xaxes(zeroline=True, zerolinewidth=4)
    mm_vs_lb.update_yaxes(zeroline=True, zerolinewidth=4)

    ## Update Overall Figure
    mm_vs_lb.update_layout(template='plotly_white',
                           title=title + " Inhibition")
    
    return(mm_vs_lb)

def mm_vs_lb_update(ki, inhib, mm_func, lb_func, graph):
    """ Updates the plots given the slider inputs
    
    Arguments:
        ki: Slider input to control K_I
        inhib: Slider input to control [I]
        mm_func: Function to evaluate the Michaelis-Menten equation
        lb_func: Function to evaluate the Lineweaver-Burk equation
        graph: Plotly widget/plot that will be updated
    """
        
    new_vel = mm_func(ki, inhib)
    graph.data[0].y = new_vel

    new_recip_vel = lb_func(ki, inhib)
    graph.data[1].y = new_recip_vel

def make_widgets():
    """ Creates the ipywidgets to control K_I and [I]
    
    Returns:
        slider_ui: Slider UI arranged in a horizontal box
        ki_widget: Slider widget controlling K_I
        inhib_widget: Slider widget controlling [I] 
    """
    
    ki_widget = widgets.IntSlider(value=50, min=1, max=100, description="$K_{I}$")
    inhib_widget = widgets.IntSlider(value=50, min=1, max=200, description="$[I]$")
    slider_ui = widgets.HBox([ki_widget, inhib_widget])
    
    return(slider_ui, ki_widget, inhib_widget)

def plot_and_display(title, mm_func, lb_func):
    """ Generates and displays graphs using above helper functions
    
    Arguments:
        title: String of the inhibition type
        mm_func: Function to evaluate the Michaelis-Menten equation
        lb_func: Function to evaluate the Lineweaver-Burk equation
    """
    
    fig = plot_inhibition(title, mm_func, lb_func)  # Generate Figure
    slider_ui, ki_widget, inhib_widget = make_widgets()  # Genereature Widgets

    ## Initializes widgets to update the graph
    widgets.interactive_output(mm_vs_lb_update, {'ki': ki_widget, 'inhib' : inhib_widget,
                                                 'mm_func': fixed(mm_func),
                                                 'lb_func' : fixed(lb_func),
                                                 'graph' : fixed(fig)})

    # Display graph and UI
    display(fig)
    display(slider_ui)
    

### Additional Logistics ### 
# Edit to control concentration range #
conc_low = -200  # Lowest concentration
conc_high = 400  # Highest Concentration
sub_neg = np.linspace(conc_low,-1, 100)
sub_pos = np.linspace(1,conc_high, 100)
substrate = np.concatenate((sub_neg, sub_pos)) # Substrate Concentration Vector

### Starting Enzyme Constants and Inhibitor Concentration###
km = 30
vmax = 20
ki = 50
inhib_conc = 50

<a id='competitive'></a>

# Competitive Inhibition

## Michaelis-Menten Equation
$$v = \frac{V_{max} [S]}{(1 + \frac{[I]}{K_I})K_m + [S]}$$

## Lineweaver-Burk Equation

$$\frac{1}{v} = \frac{1}{V_{max}} + \big(1 + \frac{[I]}{K_I}\big)\frac{K_m}{V_{max}}\frac{1}{[S]}$$

In [3]:
def mm_competitive(ki, inhib_conc):
    vel = (vmax * sub_pos) / ((1 + inhib_conc / ki) * km + sub_pos)
    return(vel)

def lb_competitive(ki, inhib_conc):
    recip_vel = 1 / vmax + (1 + inhib_conc/ki) * (km/vmax) * 1 / substrate
    return(recip_vel)

plot_and_display('Competitive', mm_competitive, lb_competitive)

FigureWidget({
    'data': [{'line': {'color': 'red'},
              'mode': 'lines',
              'showlegen…

<a id='uncompetitive'></a>

## Uncompetitive Inhibition

### Michaelis-Menten Equation

$$v = \frac{V_{max} [S]}{K_m + (1 + \frac{[I]}{K_I})[S]}$$

### Lineweaver-Burk Equation

$$\frac{1}{v} = \big(1 + \frac{[I]}{K_I}\big)\frac{1}{V_{max}} + \frac{K_m}{V_{max}}\frac{1}{[S]}$$

In [4]:
def mm_uncompetitive(ki, inhib_conc):
    vel = (vmax * sub_pos) / (km + (1 + inhib_conc / ki) * sub_pos)
    return(vel)

def lb_uncompetitive(ki, inhib_conc):
    recip_vel = (1 + inhib_conc/ki) * 1 / vmax + (km/vmax) * 1 / substrate
    return(recip_vel)

plot_and_display('Uncompetitive', mm_uncompetitive, lb_uncompetitive)

FigureWidget({
    'data': [{'line': {'color': 'red'},
              'mode': 'lines',
              'showlegen…

<a id='noncompetitive'></a>

## Noncompetitive Inhibition

### MM Equation

$$v = \frac{V_{max} [S]}{(1 + \frac{[I]}{K_I})K_m + (1 + \frac{[I]}{K_{I,2}})[S]}$$

### LB Equation

$$\frac{1}{v} = (1 + \frac{[I]}{K_{I,2}})\frac{1}{V_{max}} + (1 + \frac{[I]}{K_I})\frac{K_m}{V_{max}}\frac{1}{[S]}$$

In [5]:
def mm_noncompetitive(ki, inhib_conc):
    vel = (vmax * sub_pos) / ((1 + inhib_conc / ki) * km + (1 + inhib_conc / ki) * sub_pos)
    return(vel)

def lb_noncompetitive(ki, inhib_conc):
    recip_vel = (1 + inhib_conc/ki) * 1 / vmax + (1 + inhib_conc / ki) * (km/vmax) * 1 / substrate
    return(recip_vel)

plot_and_display('Noncompetitive', mm_noncompetitive, lb_noncompetitive)

FigureWidget({
    'data': [{'line': {'color': 'red'},
              'mode': 'lines',
              'showlegen…

<a id='key-observations'></a>

## Key Observations
Below are a list of key observations for each of the types of inhibition that you should notice:

### Competitive Inhibition
1. The inhibition can be overcome with a large enough $[S]$ (i.e., the Michaelis-Menten graph will eventually reach $V_{max}$
2. Changing $[I]$ alters the slope of the Lineweaver-Burk plot
3. Chagning $[I]$ does not alter the y-intercept ($V_{max}$)

### Uncompetitive Inhibition
1. Inhibition cannot be overcome with large $[S]$ (i.e., $V_{max}$ has decreased)
2. Changing $[I]$ does not alter the slope of the Lineweaver-Burk plot
3. Changing $[I]$ alters *both* the x- and y-intercept ($K_m$ and $V_{max}$, respectively)

### Noncompetitive Inhibition
1. Inhibition cannot be overcome with large $[S]$ (i.e., $V_{max}$ has decreased)
2. Changing $[I]$ does alters the slope of the Lineweaver-Burk plot
3. Changing $[I]$ does not alter the x-intercept ($K_m$)